In [13]:
import sys
import os
lib = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)

In [14]:
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
import pandas as pd
import os.path as path

PROJECT_PATH = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'

In [15]:
# Functions box
def dict_append(dic, plc, val): 
    dic[plc] = dic[plc] + [val]
    

def find_indexes_of_markers_in_sample(_sample_genes, markers):
    return [idx1 for idx1, g1 in enumerate(_sample_genes) for idx2, g2 in enumerate(markers) if g1==g2]


def get_all_possible_combinations_of_markers(cell_type_name, celltype_markers):
    def recursive_comnibation_building(leftover, all_combinations=[[]]):
            def meiosis_of_combination(and_markers, or_markers):
                new_combinations = []
                for or_marker in or_markers:
                    for and_marker in and_markers:
                        new_combinations.append(and_marker + [or_marker])
                return new_combinations

            if len(leftover) == 0:
                return all_combinations
            all_currrent_markers = leftover[0]
            del leftover[0]
            all_combination = meiosis_of_combination(all_combinations, all_currrent_markers)
            return recursive_comnibation_building(leftover, all_combination)
    initial_marker_structure =[s.split(';') for s in  celltype_markers]
    combinations = recursive_comnibation_building(initial_marker_structure)
    return combinations

## Extract sample from PC 

In [133]:
# Extracts one of the samples from PC
sample_id = 'M126'
sample = fr'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples\{sample_id}\RNA_sample.pkl'
data = pickle.load(open(sample, 'rb'))
sample_genes = data.gene_names
counts = data.counts.T
n_of_cells = counts.shape[0]
n_of_genes = counts.shape[0]
print(f'count shape {counts.shape}')
print(f'number of cells {n_of_cells}')
print(f'number of genes {n_of_genes}')


count shape (3125, 19571)
number of cells 3125
number of genes 3125


# Step 1 - Building mapping of cell-types with positive markers
* After this step cell_types_mapping_table object should map between each cell-index in extracted-sample's count-table and cell-type before elimination by negative markers
* Next step is to eliminate cell-types by 'None' tab in Excel file and then analays results.

In [134]:
# Extract ImmuneCellsMarkersUpdated Excel file from PC and load it into DataFrame.
MARKERS_PATH = path.join(PROJECT_PATH, r'Data\ImmuneCellsMarkersUpdated.xlsx')
xls = pd.ExcelFile(MARKERS_PATH)
df = pd.read_excel(xls, 'and_or')

# df = pd.read_excel(xls, 'Sheet1')

# Reorganize the table, the column names actually accouns the first row of the DataFrame
columns = [v if not 'Unnamed' in v else np.NAN for v in  list(df.columns)]
df.loc[28] =  columns
df = df.reindex([28]+list(range(0,28))).reset_index(drop=True)

# convert df to dictionary markers table
rm_nan = lambda lst: list([vv for vv in lst if not vv is np.NAN])
df_rows = df.values.tolist()
map_celltype_to_markers = {v[0]: rm_nan(v[1:]) for v in df_rows}

df.head(10)

,T cells,CD3E,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,T cells,CD3E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CD4 helper T cells,CD3E,CD4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CD8 Cytotoxic T cells,CD3E,CD8A;CD8B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Regulatory T cells,CD3E,FOXP3,CTLA4;IL2RA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Regulatory CD4 T cells,CD4,CD3E,FOXP3,CTLA4;IL2RA,NaN,NaN,NaN,NaN,NaN,NaN
5,Regulatory CD8 T cells,CD3E,FOXP3,CTLA4;IL2RA,CD8A;CD8B,NaN,NaN,NaN,NaN,NaN,NaN
6,Regulatory CD4_CD8 T cells,CD3E,FOXP3,CTLA4;IL2RA,CD4,CD8A;CD8B,NaN,NaN,NaN,NaN,NaN
7,NKT cells,CD3E,NCR1;NCAM1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NK cells,FCGR3A,NCR1;NCAM1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,B cells,CD19;MS4A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
# Builds mapping-table of sample's count-table

positive_cell_types_mapping_table = {v: [] for v in range(counts.shape[0])}
for curr_cell_type, curr_markers in map_celltype_to_markers.items():
    print(curr_cell_type)
    print(curr_markers)

    possible_combinations_of_markers = get_all_possible_combinations_of_markers(curr_cell_type, curr_markers)
    for curr_combination_markers in possible_combinations_of_markers:
        curr_genes_indexes = find_indexes_of_markers_in_sample(sample_genes, curr_combination_markers)
        counts_only_interesting_genes = np.take(counts, curr_genes_indexes, axis=1)
        cells_satisfying_markers = np.all(counts_only_interesting_genes > 0, axis = 1)
        cells_satisfying_markers_indexs = np.where(cells_satisfying_markers)
        print(cells_satisfying_markers_indexs[0].shape)

    
    [dict_append(positive_cell_types_mapping_table, cell, curr_cell_type) for cell in cells_satisfying_markers_indexs[0]]


T cells
['CD3E']
(1926,)
CD4 helper T cells
['CD3E', 'CD4']
(148,)
CD8 Cytotoxic T cells
['CD3E', 'CD8A;CD8B']
(878,)
(877,)
Regulatory T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA']
(36,)
(25,)
Regulatory CD4 T cells
['CD4', 'CD3E', 'FOXP3', 'CTLA4;IL2RA']
(12,)
(11,)
Regulatory CD8 T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA', 'CD8A;CD8B']
(3,)
(1,)
(5,)
(1,)
Regulatory CD4_CD8 T cells
['CD3E', 'FOXP3', 'CTLA4;IL2RA', 'CD4', 'CD8A;CD8B']
(1,)
(0,)
(1,)
(0,)
NKT cells
['CD3E', 'NCR1;NCAM1']
(22,)
(9,)
NK cells
['FCGR3A', 'NCR1;NCAM1']
(4,)
(3,)
B cells
['CD19;MS4A1']
(171,)
(325,)
Activated T cells
['CD3E', 'CD2', 'CD28', 'IL2RA;CD69;ICOS;TNFRSF4;TNFRSF9;CD27', 'IL2;TNF;IFNG', 'CD8A;CD8B;CD4']
(0,)
(1,)
(0,)
(0,)
(1,)
(1,)
(1,)
(6,)
(3,)
(1,)
(4,)
(5,)
(1,)
(15,)
(9,)
(4,)
(5,)
(16,)
(0,)
(1,)
(0,)
(0,)
(1,)
(1,)
(1,)
(3,)
(2,)
(1,)
(3,)
(3,)
(1,)
(12,)
(8,)
(4,)
(2,)
(13,)
(0,)
(0,)
(1,)
(1,)
(0,)
(1,)
(0,)
(1,)
(0,)
(1,)
(0,)
(0,)
(0,)
(2,)
(3,)
(1,)
(0,)
(4,)
Senescence T cells
['CD3E', 'KLRG

# Step 2 - Building mapping of cell-types with Negative markers

In [136]:
# Extract ImmuneCellsMarkersUpdated Excel file from PC and load it into DataFrame.
xls = pd.ExcelFile(MARKERS_PATH)
negative_markers_df = pd.read_excel(xls, 'none')

# Reorganize the table, the column names actually accouns the first row of the DataFrame
negative_markers_columns = [v if not 'Unnamed' in v else np.NAN for v in  list(negative_markers_df.columns)]
negative_markers_df.loc[28] =  negative_markers_columns
negative_markers_df = negative_markers_df.reindex([28]+list(range(0,28))).reset_index(drop=True)

# convert df to dictionary markers table
negative_df_rows = negative_markers_df.values.tolist()
negative_map_celltype_to_markers = {v[0]: rm_nan(v[1:]) for v in negative_df_rows}


In [137]:
# Builds mapping-table of sample's count-table

negative_cell_types_mapping_table = {v: [] for v in range(counts.shape[0])}
for curr_cell_type, curr_markers in negative_map_celltype_to_markers.items():
    print(curr_cell_type)
    print(curr_markers)

    curr_genes_indexes = find_indexes_of_markers_in_sample(sample_genes, curr_markers)
    counts_only_interesting_genes = np.take(counts, curr_genes_indexes, axis=1)
    cells_satisfying_markers = np.any(counts_only_interesting_genes > 0, axis = 1)
    cells_satisfying_markers_indexs = np.where(cells_satisfying_markers)
    print(cells_satisfying_markers_indexs[0].shape)

    
    [dict_append(negative_cell_types_mapping_table, cell, curr_cell_type) for cell in cells_satisfying_markers_indexs[0]]

T cells
['NCR1', 'NCAM1', 'FOXP3']
(136,)
CD4 helper T cells
['NCR1', 'NCAM1', 'FOXP3']
(136,)
CD8 Cytotoxic T cells
['NCR1', 'NCAM1', 'FOXP3']
(136,)
Regulatory T cells
['CD4', 'CD8A', 'CD8B']
(1554,)
Regulatory CD4 T cells
['CD8A', 'CD8B']
(1257,)
Regulatory CD8 T cells
['CD4']
(309,)
Regulatory CD4_CD8 T cells
[]
(0,)
NKT cells
['FOXP3']
(79,)
NK cells
['CD3E', 'FOXP3']
(1936,)
B cells
['CD3E', 'FOXP3']
(1936,)
Activated T cells
['NCR1', 'NCAM1']
(59,)
Senescence T cells
['CD28', 'CD27', 'NCR1', 'NCAM1']
(1401,)
Terminal effector
[]
(0,)
Exhausted T cells
['NCR1', 'NCAM1']
(59,)
Stem_like T cells
['FOXP3']
(79,)
Memory T cells
['CD4', 'CD8A', 'CD8B', 'FOXP3']
(1605,)
Memory CD4 T cells
['CD8A', 'CD8B', 'FOXP3']
(1330,)
Memory CD8 T cells
['CD4', 'FOXP3']
(365,)
Memory CD4_CD8 T cells
['FOXP3']
(79,)
Macrophage_immature
['HLA-DRA', 'CD3E']
(2591,)
Macrophage_mature
['CD3E']
(1926,)
Monocyte_immature
['HLA-DRA']
(1060,)
Monocyte_mature
['CD3E']
(1926,)
cDCs_dendritic_cells
['CD3E']
(1

In [138]:
# for i in range(20):
#     print(negative_cell_types_mapping_table[i])
#     print(cell_types_mapping_table[i])
#     print('\n')

# Cross-checks determinations between neg-markers and pos-markers and finds conflicts

In [139]:
n_overlapping = 0
n_cls = 0

for i in range(n_of_cells):
    pos_celltypes = positive_cell_types_mapping_table[i]
    neg_celltypes = negative_cell_types_mapping_table[i]
    overlapping = intersection_of_lists(pos_celltypes, neg_celltypes)
    if len(pos_celltypes)>0:
        n_cls += 1
    if len(overlapping)>0:
        n_overlapping += 1
#         print(i)
#         print(pos_celltypes)
#         print(neg_celltypes)
print(n_overlapping)
print(n_cls)
print(n_of_cells)
print(n_cls/n_of_cells)

221
2333
3125
0.74656


In [140]:
positive_cell_types_mapping_table

{0: ['B cells'],
 1: [],
 2: ['T cells', 'CD8 Cytotoxic T cells'],
 3: ['T cells'],
 4: [],
 5: ['T cells', 'CD8 Cytotoxic T cells'],
 6: ['T cells', 'CD8 Cytotoxic T cells'],
 7: ['T cells'],
 8: [],
 9: ['T cells'],
 10: ['T cells'],
 11: ['T cells', 'CD8 Cytotoxic T cells'],
 12: [],
 13: ['myeloid cells_general_immature', 'myeloid cells_general_mature'],
 14: ['T cells', 'CD8 Cytotoxic T cells'],
 15: [],
 16: ['T cells'],
 17: [],
 18: ['myeloid cells_general_immature', 'myeloid cells_general_mature'],
 19: ['T cells'],
 20: ['T cells'],
 21: [],
 22: ['Monocyte_immature', 'Monocyte_mature'],
 23: ['T cells', 'CD8 Cytotoxic T cells'],
 24: [],
 25: ['T cells'],
 26: ['B cells'],
 27: ['B cells'],
 28: [],
 29: [],
 30: ['T cells'],
 31: [],
 32: ['T cells', 'CD8 Cytotoxic T cells', 'NKT cells'],
 33: [],
 34: [],
 35: ['T cells'],
 36: [],
 37: ['T cells'],
 38: [],
 39: ['B cells'],
 40: ['T cells'],
 41: [],
 42: ['T cells', 'CD8 Cytotoxic T cells'],
 43: ['T cells'],
 44: ['T c

In [ ]:
# immune_genes = ['PTPRC', 'CD3E', 'CD8A', 'CD8B', 'CD4', 'MS4A1', 'CD163', 'CD14', 'CD19', 'CD2', 'NCR1', 'CD1C'] # immune general
# macrophage_genes = ['CD14', 'CD163'] # macrophages
# DC_genes = ['LILRA4', 'CD1C', 'CLE4C'] # dendritic cells
# lym_genes = ['CD3E','CD8A', 'CD8B', 'MS4A1', 'CD19','CD2', 'NCR1'] # lympocytes
# neutrophil_genes = ['FCGR3B', 'CXCR1','CXCR2','G0S2','CSF3R'] # neutrophils
# tumor_genes = ['MLANA', 'PMEL','TYR','MITF','AXL'] # tumor genes

# for gene in immune_genes:
#     if not gene in genes:
#         print(gene)
# immune_gene_indexes = [idx1 for idx1, g1 in enumerate(sample_genes) for idx2, g2 in enumerate(immune_genes) if g1==g2]
# sample_genes[immune_gene_indexes[0]]
# len(immune_gene_indexes)
# # counts.shape[:, immune_gene_indexes]
# interest_region = np.take(counts, immune_gene_indexes, axis=1)
# sum(np.sum(interest_region == 0, axis = 1)==0)


In [ ]:
# Debug zone!!!

# ROOT_PATH = r'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples'
df_doc = pd.DataFrame(columns=['file', 'number of cells', 'number of classified cells', 'portion in percenatge'])
folders = [subfolder for subfolder in os.listdir(ROOT_PATH)]
folders
for sample_id in folders:
    print(sample_id)
    sample = fr'C:\Users\itay\Desktop\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\Data\rna_seq200k\all_samples\{sample_id}\RNA_sample.pkl'
    data = pickle.load(open(sample, 'rb'))
    sample_genes = data.gene_names
    counts = data.counts.T
    n_of_cells = counts.shape[0]
    n_of_genes = counts.shape[0]
#     print(f'count shape {counts.shape}')
    print(f'number of cells {n_of_cells}')
#     print(f'number of genes {n_of_genes}')
    len([v for v in positive_cell_types_mapping_table.values() if len(v)>0])

    curr_genes_indexes = find_indexes_of_markers_in_sample(sample_genes, ['CD3E', 'CD163', 'FUT4'])
    counts_only_interesting_genes = np.take(counts, curr_genes_indexes, axis=1)
    cells_satisfying_markers = np.any(counts_only_interesting_genes > 0, axis = 1)
    cells_satisfying_markers_indexs = np.where(cells_satisfying_markers)
    n_c_c = cells_satisfying_markers_indexs[0].shape[0]
    print(f'number of classified cells: {n_c_c}')
    df_doc = df_doc.append({'file':sample_id,  'number of cells':n_of_cells, 'number of classified cells':n_c_c, 'portion in percenatge':n_c_c/n_of_cells}, ignore_index=True)    # cells_satisfying_markers_indexs
    df_doc.to_csv(r'C:\Users\itay\Desktop\test\o.csv')